# Chatbot com LangGraph

## Configuração

Carregue variáveis de ambiente

In [ ]:
from dotenv import load_dotenv
_ = load_dotenv()

## Modelo

Configure o LangChain para usar o LLM desejado.

In [ ]:
# from langchain_ollama import ChatOllama

# llm = ChatOllama(
#     model="llama3.2", 
#     temperature=0.0, 
#     max_tokens=2000,
# )

In [ ]:
# from langchain_cerebras import ChatCerebras
# llm = ChatCerebras(
#     model="llama-3.3-70b",
#     temperature=0.0,
#     max_tokens=2000,
# )

In [ ]:
# from langchain_groq import ChatGroq
# llm = ChatGroq(
#     model="qwen-qwq-32b", 
#     temperature=0.0, 
#     max_tokens=2000,
# )

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama3-70b-8192", 
    temperature=0.0, 
    max_tokens=2000,
)

In [ ]:
# from langchain_groq import ChatGroq
# llm = ChatGroq(
#     model="llama3-8b-8192", 
#     temperature=0.0, 
#     max_tokens=2000,
# )

## Ferramentas

Ferramentas externas usadas pelo agente.

In [ ]:
tools = []

Ferramenta de busca na internet:

In [ ]:
# from langchain_community.tools.tavily_search import TavilySearchResults

# tools.append(TavilySearchResults(max_results=2))

Ferramenta de busca FAKE:

In [ ]:
from langchain_core.tools import tool


@tool
def search(query):
    """A search tool used to get information.

    Args:
        query: The query to search for.
        
    Returns:
        str: The result of the search."""
    
    if "campinas" in query.lower():
        return "Em 2022, a população de Campinas foi estimada pelo Instituto Brasileiro de Geografia e Estatística em 1.139.047 habitantes."
    elif "são paulo" in query.lower():
        return "A cidade de São Paulo tem uma população estimada de 11.895.578 habitantes em 2024. Essa população faz de São Paulo a maior cidade do Brasil e uma das mais populosas do mundo. "
    else:
        return "Informação não encontrada. Tente consultar de outra forma."

tools.append(search)

Calculadora:

In [ ]:
import re
from langchain_core.tools import tool


@tool
def calculate(expression):
    """A calculator tool that can perform basic arithmetic operations. Use this when you need to compute mathematical expressions or solve numerical problems.

    Args:
        expression: The mathematical expression to evaluate.
        
    Returns:
        str: The result of the calculation."""

    expression = re.sub(r'[^0-9+\-*/().]', '', expression)
    
    try:
        result = eval(expression)
        return str(result)
    except (SyntaxError, ZeroDivisionError, NameError, TypeError, OverflowError):
        return "Error: Invalid expression"

tools.append(calculate)


## Chatbot

Defina o template do prompt de sistema do chatbot

In [ ]:
prompt_template = """Você é uma assistente capaz de resolver problemas usando as ferramentas disponíveis.
Resolva o problema apresentado passo-a-passo.
No final, dê a resposta ao problema de forma concisa e completa.
"""


def make_system_prompt():
    return prompt_template

Defina o código do chatbot

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.messages import HumanMessage, SystemMessage

from IPython.display import Image, display

class ChatbotState(TypedDict):
    messages: Annotated[list, add_messages]


llm_with_tools = llm.bind_tools(tools)

tool_node = ToolNode(tools=tools)


def chatbot_node(state: ChatbotState):
    prompt = [SystemMessage(make_system_prompt())] + state["messages"]
    response = llm_with_tools.invoke(prompt)
    return {"messages": [response]}


builder = StateGraph(ChatbotState)
builder.add_node("chatbot", chatbot_node)
builder.add_node("tools", tool_node)

builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")

builder.set_entry_point("chatbot")

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

# display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

Código para facilitar o uso do chatbot:

In [ ]:
def chatbot(message: str, session_id="1"):
    config = {"recursion_limit": 10, "configurable": {"thread_id": session_id}}
    messages = {"messages": [HumanMessage(content=message)]}
    index = -1
    for event in graph.stream(messages, config=config, stream_mode="values"):
        for msg in event["messages"][index:]:
            msg.pretty_print()
            print("\n")
        index = len(event["messages"])

def clear_memory():
    memory.storage.clear()

def show_history(session_id="1"):
    config = {"configurable": {"thread_id": session_id}}
    for msg in graph.get_state(config).values["messages"]:
        msg.pretty_print()
        print("\n")

## Conversando com o chatbot

In [ ]:
clear_memory()
chatbot("Qual a população atual de Campinas?")

In [ ]:
clear_memory()
chatbot("Quanto é o dobro de 3.14?")

In [ ]:
clear_memory()
chatbot("Quanto é a razão entre a população da cidade de São Paulo e a população de Campinas?")